In [ ]:
#this line is needed when running the code in Google Colab because otherwise the data.table package is not going to work
system2('sudo', 'apt-get install libgsl0-dev')

install.packages("data.table")
install.packages("tm")
install.packages("topicmodels")
install.packages("tidytext")
install.packages("gutenbergr")
install.packages("reshape2")
library(gutenbergr)
library(curl)
library(dplyr)
library(ggplot2)
library(data.table)
library(tm)
library(topicmodels)
library(tidytext)
library(tidyr)
library(stringr)
library(scales)
library(reshape2)
library(rmarkdown)
library(tidyverse)

In [ ]:
books <- read.csv("books.csv")

by_words <- books %>%
  unnest_tokens(word, Summary)


by_words

In [ ]:
write.csv(by_words,'SummaryWords.csv')

In [ ]:
names <- read.csv("names.csv")
names$name <- tolower(names$name)
names

In [ ]:
#additional stopwords to remove
additionalWords <- read.csv("Remove.csv")

additionalWords$Words <- tolower(additionalWords$Words)

additionalWords

In [ ]:
word_counts <- by_words %>%
  filter(!word %in% c(stop_words$word, names$name, additionalWords$Words)) %>%
  count(Title, word, sort = TRUE) %>%
  ungroup()

word_counts

In [ ]:
write.csv(word_counts,'SummaryWordcounts.csv')

In [ ]:
word_counts = word_counts %>% bind_tf_idf(word, Title, n) %>%
  arrange(desc(tf_idf))


avg = mean(word_counts$tf_idf)


word_counts = subset(word_counts, tf_idf>avg)

In [ ]:
word_counts

In [ ]:
write.csv(word_counts,'SummaryWordcountsAfterTFIDF.csv')

In [ ]:
chapters_dtm <- word_counts %>%
  cast_dtm(Title, word, n)

chapters_dtm

In [ ]:
chapters_lda <- LDA(chapters_dtm, k = 9, control = list(seed = 10))
chapters_lda

In [ ]:
topics <- tidy(chapters_lda, matrix = "beta")
topics

In [ ]:
write.csv(topics,'SummaryTopics.csv')

In [ ]:
#top5 words for each of the topics
top_terms <- topics %>%
  group_by(topic) %>%
  top_n(5, beta) %>%
  ungroup() %>%
  arrange(topic, -beta)

top_terms

In [ ]:
write.csv(top_terms,'SummaryTopTermsbyTopic.csv')

In [ ]:
#visualisation for top words per topic
png(file="topWordsPerTopic.png" ,width=4000,height=4000, res = 300, bg="white")
top_terms %>%
  mutate(term = reorder_within(term, beta, topic)) %>%
  ggplot(aes(beta, term, fill = factor(topic))) +
  geom_col(show.legend = FALSE) +
  facet_wrap(~ topic, scales = "free") +
  scale_y_reordered()
dev.off()

In [ ]:
chapters_gamma <- tidy(chapters_lda, matrix = "gamma")
chapters_gamma


chapters_gamma <- chapters_gamma %>%
  separate(document, c("title", "chapter"), sep = "_", convert = TRUE)

chapters_gamma

In [ ]:
chapter_classifications <- chapters_gamma %>%
  group_by(title, chapter) %>%
  slice_max(gamma) %>%
  ungroup()

chapter_classifications

In [ ]:
book_topics <- chapter_classifications %>%
  count(title, topic) %>%
  group_by(title) %>%
  top_n(1, n) %>%
  ungroup() %>%
  transmute(consensus = title, topic)

In [ ]:
names(book_topics)[1] <- "title"
names(book_topics)[2] <- "LDAtopics"

write.csv(book_topics, 'LDAtopics.csv')